astro_101 placeholder for (course name)

# Actividad astro_101
Este cuaderno le ayudará a instalar todo lo que necesita para realizar la reducción y análisis de los datos de [...].

Asegúrese de leerlo bien.

# 1 Setup
Lo primero que debe realizar es descargar los archivos necesarios para la actividad. Para esto, debe:


1.   enlazar GD al colab
2.   descargar archivos a su GD


El primer paso que debe tomar es conectar su Google Drive (hagalo con el asociado a su correo mail.udp.cl). Para esto, en el lado izquierdo aprete el ícono que tiene forma de carpeta. En el menú que se abre, haga click en el ícono de Google Drive (carpeta con un símbolo parecido al del reciclaje) y de permiso para su conexión.

Alternativamente, ejecute el código siguiente (haciendo click sobre el icono play).

In [ ]:
import os
from google.colab import drive

In [ ]:
# conectar drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Cambiamos el directorio de trabajo al de Google Drive.
os.chdir('drive/MyDrive')

In [ ]:
#Creamos un directorio donde vamos a guardar todas las rutinas y los archivos.
!mkdir astro_101
os.chdir('astro_101')

In [ ]:
#Descargamos las rutinas desde el repositorio de Github.
!git clone https://github.com/ReddTea/udp_astro_101
#!git pull https://github.com/ReddTea/udp_astro_101

Cloning into 'udp_astro_101'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 15 (delta 1), reused 11 (delta 0), pack-reused 0
Receiving objects: 100% (15/15), 6.55 KiB | 609.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.


# 2 TEST
Nos aseguraremos que las rutinas pueden correr sin problemas con el siguiente test. Para esto es necesario:


1.   Ubicarse en la carpeta de trabajo
2.   Instalar las librerias necesarias
3.   Correr los test



In [ ]:
os.chdir('udp_astro_101')

In [ ]:
%%capture
import numpy as np
try:
    import reddemcee
except ModuleNotFoundError:
    !pip3 install git+https://github.com/ReddTea/reddemcee.git
    import reddemcee

try:
    import astroemperor as emp
except ModuleNotFoundError:
    !pip3 install git+https://github.com/ReddTea/astroemperor.git
    import astroemperor as emp

In [ ]:
def log_like(x, ivar):
    return -0.5 * np.sum(ivar * x ** 2)

def log_prior(x):
    return 0.0

ndim, nwalkers = 5, 100
ntemps = 5
ivar = 1. / np.random.rand(ndim)
p0 = list(np.random.randn(10, nwalkers, ndim))

In [ ]:
# init sampler
sampler = reddemcee.PTSampler(nwalkers, ndim, log_like, log_prior,
                              ntemps=ntemps, adaptative=True, logl_args=[ivar])

In [ ]:
s0 = sampler.run_mcmc(p0, 200, 2)

100%|██████████| 400/400 [00:06<00:00, 63.31it/s]


#

In [ ]:
sim = emp.Simulation()
sim.set_engine('reddemcee')
setup = np.array([3, 50, 250, 2])

sim.load_data('51Peg')  # Target folder name in /datafiles/
sim.plot_all['paper_mode'] = True

                                                                                
                   ~~ Simulation Successfully Initialized ~~                    
                                                                                


                         Reading data from 51peg.vels
                          




In [ ]:
sim.run_auto(setup, k_start=1, k_end=1)

                                       Offset block added, OffsetBlock          


                                       Jitter block added, JitterBlock          


                                   Keplerian block added, KeplerianBlock 1      





                                ~~ Setup Info ~~                                

Current Engine is            reddemcee 0.6

Number of cores is           2

Save location is             datalogs/51Peg/run_1/k1

Dynamical Criteria is        None

Posterior fit method is      Gaussian Mixtures

Limits constrain method is   sigma

Model Selection method is    BIC


                           ~~ Automatically Saving ~~                           


Logger       : ✔

Samples      : ✘

Posteriors   : ✔

Likelihoods  : ✔


Plots: Posteriors           : ✔

Plots: Keplerian Model      : ✔

Plots: Gaussian Mixture     : ✔

Plots: Parameter Histograms : ✔



                               ~~ Pre-Run Info ~~                               



Paramete

In [ ]:
get_ipython().__class__.__name__

'Shell'

In [ ]:
def in_ipynb():
    try:
        cfg = get_ipython().config
        if cfg['IPKernelApp']['parent_appname'] == 'ipython-notebook':
            return True
        else:
            return False
    except NameError:
        return False

In [ ]:
in_ipynb()

False

In [ ]:
get_ipython().__class__.__module__

'google.colab._shell'

In [ ]:
def my_shell_env():
    try:
        #
        shl = get_ipython().__class__.__name__
        if shl == 'ZMQInteractiveShell':
            return 'jupyter-notebook'
        elif shl == 'TerminalInteractiveShell':
            return 'ipython-terminal'
        elif get_ipython().__class__.__module__ == 'google.colab._shell':
            return 'google-colab'

    except NameError:
        return 'python-terminal'

In [ ]:
get_ipython().config

{'ColabKernelApp': {'connection_file': '/root/.local/share/jupyter/runtime/kernel-03ed6b5a-b621-43b7-a340-a366cd32a699.json'},
 'IPKernelApp': {'kernel_class': 'google.colab._kernel.Kernel',
  'parent_appname': <LazyConfigValue {}>},
 'Completer': {'use_jedi': False},
 'InteractiveShellApp': {'extensions': ['google.colab'],
  'reraise_ipython_extension_failures': True,
  'exec_lines': ['__import__("logging").basicConfig()',
   '__import__("logging").getLogger("google.auth._default").setLevel(40)',
   '__import__("logging").getLogger("googleapiclient.discovery_cache").setLevel(40)'],
  'matplotlib': 'inline'},
 'HistoryManager': {'hist_file': ':memory:'}}

In [ ]:
get_ipython().__class__.__name__

'Shell'

In [ ]:
get_ipython().__class__.__module__

'google.colab._shell'

In [ ]:
my_shell_env()

'google-colab'